<center><b>Chatura Samarasinghe<br> ELU, He innitialization, early stopping and transfer learning</b><center>

In [1]:
import tensorflow as tf
import numpy as np

load data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./tmp/data/')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz


In [60]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

hyperparameters

In [61]:
n_input = 28*28
n_output_p1 = 10

learning_rate = 0.01

#neurons
n_hidden = 100


<b>Construction Phase</b>

In [105]:
reset_graph()

X = tf.placeholder(name="X", dtype=tf.float32, shape=(None, n_input))
y = tf.placeholder(name="y", dtype=tf.int32, shape=(None))

In [106]:
import tensorflow.contrib.layers as cl

In [107]:
''' 
below code is same as doing 
hidden1 = cl.fully_connected(X, n_hidden1, weights_initializer=he_init, activation_fn=tf.nn.elu)
for all the layers 
'''
# he initialization
he_init = cl.variance_scaling_initializer()
    
hidden1 = tf.layers.dense(X, n_hidden, name = "hidden1", activation = tf.nn.elu, kernel_initializer = he_init)
hidden2 = tf.layers.dense(hidden1, n_hidden, name = "hidden2", activation = tf.nn.elu, kernel_initializer = he_init)
hidden3 = tf.layers.dense(hidden2, n_hidden, name = "hidden3", activation = tf.nn.elu, kernel_initializer = he_init)
hidden4 = tf.layers.dense(hidden3, n_hidden, name = "hidden4", activation = tf.nn.elu, kernel_initializer = he_init)
hidden5 = tf.layers.dense(hidden4, n_hidden, name = "hidden5", activation = tf.nn.elu, kernel_initializer = he_init)

logits = tf.layers.dense(hidden5, n_output_p1, name = "logits", kernel_initializer = he_init)
Y_proba = tf.nn.softmax(logits, name = "Y_proba")

loss / cost function

In [108]:

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
loss = tf.reduce_mean (cross_entropy, name="loss")

train

In [109]:
with tf.name_scope("train"):    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate)

    training_op = optimizer.minimize(loss)

evaluation

In [110]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [111]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

<b> Execution Phase </b>

In [112]:
n_epochs = 100
batch_size = 20

In [113]:
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]

In [116]:
max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        
        n_batches = len(X_train1) // batch_size

        for rnd_indices in np.array_split(rnd_idx, n_batches):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

        # using early stopping approach
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid1, y: y_valid1})
        
        if loss_val < best_loss:
            save_path = saver.save(sess,"./mnist_trained_0to4")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
            
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

        
with tf.Session() as sess:
    saver.restore(sess, "./mnist_trained_0to4")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.128046	Best loss: 0.128046	Accuracy: 96.68%
1	Validation loss: 0.107482	Best loss: 0.107482	Accuracy: 96.83%
2	Validation loss: 0.289362	Best loss: 0.107482	Accuracy: 92.38%
3	Validation loss: 0.452124	Best loss: 0.107482	Accuracy: 85.11%
4	Validation loss: 0.389501	Best loss: 0.107482	Accuracy: 85.73%
5	Validation loss: 0.246087	Best loss: 0.107482	Accuracy: 93.75%
6	Validation loss: 0.174858	Best loss: 0.107482	Accuracy: 97.81%
7	Validation loss: 0.136970	Best loss: 0.107482	Accuracy: 97.38%
8	Validation loss: 0.126417	Best loss: 0.107482	Accuracy: 98.28%
9	Validation loss: 0.205518	Best loss: 0.107482	Accuracy: 96.52%
10	Validation loss: 0.828869	Best loss: 0.107482	Accuracy: 74.35%
11	Validation loss: 1.727138	Best loss: 0.107482	Accuracy: 20.91%
12	Validation loss: 1.766021	Best loss: 0.107482	Accuracy: 22.01%
13	Validation loss: 1.961557	Best loss: 0.107482	Accuracy: 20.91%
14	Validation loss: 1.649680	Best loss: 0.107482	Accuracy: 19.27%
15	Validation loss: 

INFO:tensorflow:Restoring parameters from ./mnist_trained_0to4


Final test accuracy: 97.43%


<b>Transfer Learning<b>

In [117]:
reset_graph()

# restore model
restore_saver = tf.train.import_meta_graph("./mnist_trained_0to4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")

freezing lower layers

In [119]:
learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

In [121]:
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

data for mnist digits 5 to 9

In [122]:
# We need to subtract 5 from the labels because TensorFlow expects integers from 0 to n_classes-1.

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

keep only 100 instances per class

In [123]:
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

In [124]:
X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

training model

In [125]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./mnist_trained_0to4")
    for var in output_layer_vars:
        var.initializer.run()

    t0 = time.time()
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./mnist_trained_0to4


INFO:tensorflow:Restoring parameters from ./mnist_trained_0to4


0	Validation loss: 1.564859	Best loss: 1.564859	Accuracy: 42.67%
1	Validation loss: 1.379052	Best loss: 1.379052	Accuracy: 42.67%
2	Validation loss: 1.479223	Best loss: 1.379052	Accuracy: 39.33%
3	Validation loss: 1.478531	Best loss: 1.379052	Accuracy: 41.33%
4	Validation loss: 1.398207	Best loss: 1.379052	Accuracy: 36.67%
5	Validation loss: 1.379148	Best loss: 1.379052	Accuracy: 42.00%
6	Validation loss: 1.369241	Best loss: 1.369241	Accuracy: 43.33%
7	Validation loss: 1.398113	Best loss: 1.369241	Accuracy: 44.67%
8	Validation loss: 1.434817	Best loss: 1.369241	Accuracy: 36.67%
9	Validation loss: 1.376881	Best loss: 1.369241	Accuracy: 44.00%
10	Validation loss: 1.480938	Best loss: 1.369241	Accuracy: 43.33%
11	Validation loss: 1.380203	Best loss: 1.369241	Accuracy: 44.67%
12	Validation loss: 1.368326	Best loss: 1.368326	Accuracy: 46.00%
13	Validation loss: 1.413625	Best loss: 1.368326	Accuracy: 42.67%
14	Validation loss: 1.436285	Best loss: 1.368326	Accuracy: 41.33%
15	Validation loss: 

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_five_frozen


Final test accuracy: 44.58%


<b>Conclusion</b><br> The low test accuracy is assumed to be a low score due to the small dataset.